In [ ]:
import easyocr
from ultralytics import YOLO
import cv2
import numpy as np
from collections import defaultdict

In [ ]:
model = YOLO("../notebooks/runs/detect/train/weights/best.pt")

reader = easyocr.Reader(['en'])

In [ ]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)
    for detection in detections:
        bbox, text, score = detection
        print(f"bbox: {bbox}, text: {text}, score: {score}")
        text = text.upper().replace(' ', '').replace('_', '').replace('-', '').replace('.', '').replace(';', '').replace(',', '').replace(':', '').replace('?', '').replace('!', '').replace('*', '').replace('(', '').replace(')', '').replace("'", "").replace('"', '')
        return bbox, text, score

    return None, None, None

In [ ]:
def areaFilter(minArea, inputImage):
    componentsNumber, labeledImage, componentStats, componentCentroids = \
        cv2.connectedComponentsWithStats(inputImage, connectivity=4)

    remainingComponentLabels = [i for i in range(1, componentsNumber) if componentStats[i][4] >= minArea]

    filteredImage = np.where(np.isin(labeledImage, remainingComponentLabels) == True, 255, 0).astype('uint8')

    return filteredImage

In [ ]:
frame = cv2.imread('{IMAGE_PATH}')
license_plates = model(frame)[0]
license_plate_text = None

BINARY_THRESH = [100, 120, 150, 170, 190]
LAST_PARAM = [cv2.THRESH_BINARY, cv2.THRESH_BINARY_INV]
prediction_dict = defaultdict(lambda: [set(), 0, []])

cv2.imshow('image', frame)
cv2.waitKey(0)
for license_plate in license_plates.boxes.data.tolist():
    x1, y1, x2, y2, score, class_id = license_plate
    license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]
    cv2.imshow('license_plate', license_plate_crop)
    cv2.waitKey(0)
    license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
    cv2.imshow('license_plate_thresh', license_plate_crop_gray)
    cv2.waitKey(0)

    for last_param in LAST_PARAM:
        for thresh in BINARY_THRESH:
            _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, thresh, 255, last_param)
            cv2.imshow('license_plate_thresh', license_plate_crop_thresh)
            cv2.waitKey(0)

            minArea = 100
            binaryImage = areaFilter(minArea, license_plate_crop_thresh)
            cv2.imshow("Filtered", binaryImage)
            cv2.waitKey(0)

            kernelSize = 3
            opIterations = 2
            morphKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernelSize, kernelSize))
            binaryImage = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, morphKernel, None, None, opIterations, cv2.BORDER_REFLECT101)
            cv2.imshow("Filtered_ultra", binaryImage)
            cv2.waitKey(0)


            _, license_plate_text, score_plate = read_license_plate(binaryImage)
            if license_plate_text is None:
                continue
            prediction_dict[license_plate_text][0] = license_plate_text
            prediction_dict[license_plate_text][1] +=1
            prediction_dict[license_plate_text][2].append(score_plate)

cv2.destroyAllWindows()


for key in prediction_dict:
    prediction_dict[key][2] = sum(prediction_dict[key][2]) / len(prediction_dict[key][2])

result = dict(prediction_dict)
for k, v in result.items():
    print(f"Set: {v[0]}, Count: {v[1]}, Mean Score: {v[2]}")